In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.spatial import ConvexHull
import scanpy.external as sce
import scanpy as sc
import scipy

from importlib import reload

# locals
import utils as ut
import plotting as plt2

# Load iHSC data

In [2]:
fpath = "/scratch/indikar_root/indikar1/cstansbu/HSC/scanpy/analysis.anndata.h5ad"

adata = sc.read_h5ad(fpath)
adata.var_names = adata.var['gene_name'].values
adata

AnnData object with n_obs × n_vars = 8563 × 15065
    obs: 'n_genes', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'EC_score', 'EB_score', 'FB_score', 'HSC_score', 'MK_score', 'new_cluster', 'cluster_str'
    var: 'gene_name', 'Chromosome', 'Start', 'End', 'Strand', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'ensembl_id', 'tf', 'bj', 'hsc', 'EC', 'EB', 'FB', 'HSC', 'MK', 'cluster_genes'
    uns: 'HSC_vs_FB', 'HSC_vs_FB_pure', 'clusters', 'deg', 'fb_vs_hsc_up', 'go_annotations', 'hsc_v_fib_up', 'hvg', 'log1p', 'neighbors', 'new_cluster', 'new_cluster_colors', 'panglaodb', 'pca', 'scenic_transcription_factors', 'scrublet', 'tabula_sapiens_deg', 'umap', 'v5_tags'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'filtered_counts', 'raw_counts'
    obsp: 'connectivities', 'distances'

# Load candidates

In [3]:
fpath = "../resources/spiked_in_candidates.csv"

df = pd.read_csv(fpath)
df.head()

,barcode
0,AAACCCAGTACCTAAC
1,AAACGAACAGCGAACA
2,AAAGGATAGCCACAAG
3,AAAGGATGTCTCAAGT
4,AAAGGGCAGCGCGTTC


In [4]:
adata.obs['candidate'] = adata.obs.index.isin(df['barcode'].values)
adata.obs['candidate'].value_counts()

candidate
False    7381
True     1182
Name: count, dtype: int64

In [5]:
adata.obs[['cluster_str', 'candidate' ]].value_counts().reset_index().sort_values(by='cluster_str')

,cluster_str,candidate,count
0,C1,False,3411
3,C1,True,516
1,C2,False,1571
7,C2,True,181
2,C3,False,1415
6,C3,True,293
4,C4,False,509
9,C4,True,93
5,C5,False,374
10,C5,True,73


# save the results

In [6]:
codes = adata.obs.copy()

columns = [
    'cell_id',
    'predicted_doublet',
    'cluster_str',
    'candidate'
]

codes = codes.reset_index()
codes[columns].head()

,cell_id,predicted_doublet,cluster_str,candidate
0,AAACCCAAGGTTACCT,False,C1,False
1,AAACCCAAGTTGAAGT,False,C1,False
2,AAACCCAAGTTGTCGT,False,C4,False
3,AAACCCACAGAAGCGT,False,C2,False
4,AAACCCACAGGAGGTT,False,C1,False


# compile gene annotations

In [7]:
genes = [
    'CD34',
    'GATA2',
    'GFI1B',
    'FOS',
    'STAT5A',
    'REL',
]

gdf = adata[:, genes].to_df()
gdf = gdf.astype(bool)
gdf = gdf.reset_index()
gdf.head()

,cell_id,CD34,GATA2,GFI1B,FOS,STAT5A,REL
0,AAACCCAAGGTTACCT,True,True,True,False,True,False
1,AAACCCAAGTTGAAGT,False,False,False,False,False,False
2,AAACCCAAGTTGTCGT,False,True,False,False,False,False
3,AAACCCACAGAAGCGT,True,False,True,False,False,False
4,AAACCCACAGGAGGTT,True,True,False,False,False,False


In [8]:
outpath = "../resources/qc_barcodes.csv"

codes = pd.merge(
    codes[columns],
    gdf,
)

codes.to_csv(outpath, index=False,)

codes.head()


,cell_id,predicted_doublet,cluster_str,candidate,CD34,GATA2,GFI1B,FOS,STAT5A,REL
0,AAACCCAAGGTTACCT,False,C1,False,True,True,True,False,True,False
1,AAACCCAAGTTGAAGT,False,C1,False,False,False,False,False,False,False
2,AAACCCAAGTTGTCGT,False,C4,False,False,True,False,False,False,False
3,AAACCCACAGAAGCGT,False,C2,False,True,False,True,False,False,False
4,AAACCCACAGGAGGTT,False,C1,False,True,True,False,False,False,False


In [9]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)